In [ ]:
import re
import numpy as np
import pickle
import os

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Đường dẫn tới thư mục bạn muốn chuyển đến
path = '/content/drive/MyDrive/DACN3_Spell_VN/'

# Thực hiện việc chuyển đến thư mục
os.chdir(path)

Mounted at /content/drive


In [ ]:
!ls model

model_spell_ngram_3.h5	model_spell_ngram_5.h5	spell_0.9899.h5  spell_ngram_3.h5
model_spell_ngram_4.h5	spell_0.9897.h5		spell_0.99.h5	 spell_ngram_5.h5


In [ ]:
from collections import Counter
from keras.models import load_model
from nltk.tokenize import word_tokenize
from nltk import ngrams, word_tokenize
import numpy as np
import re
import unidecode
import string
from tqdm import tqdm

import sys

if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")

model = load_model("model/model_spell_ngram_3.h5")

NGRAM = 3
MAXLEN = 24
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
            's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7',
            '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò',
            'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê',
            'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị',
            'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ',
            'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È',
            'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í',
            'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
letters = list(
    "abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
accepted_char = list((string.digits + ''.join(letters)))


def extract_phrases(text):
    pattern = r'\w[\w ]*\w|\s\W+|\W+|\n'
    return re.findall(pattern, text)


def encoder_data(text, maxlen=MAXLEN):
    text = "\x00" + text
    x = np.zeros((maxlen, len(alphabet)))
    for i, c in enumerate(text[:maxlen]):
        x[i, alphabet.index(c)] = 1
    if i < maxlen - 1:
        for j in range(i + 1, maxlen):
            x[j, 0] = 1
    return x


def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)


def nltk_ngrams(words, n=5):
    return ngrams(words.split(), n)


def guess(ngram):
    # Example: ngram = ('Tôi', 'ddi', 'học', 'ở', 'đâu')
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoder_data(text)]), verbose=0)
    return decoder_data(preds[0]).strip('\x00')


def correct(sentence):
    # Lặp qua từng ký tự trong câu.
    for i in sentence:
        # Kiểm tra xem ký tự hiện tại có trong danh sách các ký tự được chấp nhận hay không.
        # Nếu không, thay thế ký tự đó bằng khoảng trắng.
        if i not in accepted_char:
            sentence = sentence.replace(i, " ")

    # Kiểm tra số từ trong câu
    num_words = len(sentence.split())
    # Nếu số từ không đủ để tạo n-gram, trả về câu không đổi
    if num_words < NGRAM:
      ngrams = list(nltk_ngrams(sentence, n=num_words))
      # Đoán các từ trong các n-gram.
      guessed_ngrams = list(guess(ngram) for ngram in ngrams)
      # Tạo một danh sách các bảng ứng viên từ.
      candidates = [Counter() for _ in range(len(guessed_ngrams) + num_words - 1)]
      candidates_len = len(candidates)
      # Lặp qua các n-gram đã đoán.
      for nid, ngram in (enumerate(guessed_ngrams)):
          # Tách các từ từ n-gram.
          for wid, word in (enumerate(re.split(' +', ngram))):
              # Cập nhật bảng ứng viên từ với từ hiện tại.
              if (candidates_len>nid + wid):
                candidates[nid + wid].update([word])
      # Xây dựng câu đúng bằng cách chọn từ phổ biến nhất từ mỗi bảng ứng viên từ.
      #output = ' '.join(c.most_common(1)[0][0] for c in candidates)
      output = ' '.join(c.most_common(1)[0][0] if c else sentence.split()[index] for index, c in enumerate(candidates))
    else:
      # Tạo các n-gram từ câu đã được xử lý.
      ngrams = list(nltk_ngrams(sentence, n=NGRAM))
      # Đoán các từ trong các n-gram.
      guessed_ngrams = list(guess(ngram) for ngram in ngrams)
      # Tạo một danh sách các bảng ứng viên từ.
      candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
      candidates_len = len(candidates)
      # Lặp qua các n-gram đã đoán.
      for nid, ngram in (enumerate(guessed_ngrams)):
          # Tách các từ từ n-gram.
          for wid, word in (enumerate(re.split(' +', ngram))):
              # Cập nhật bảng ứng viên từ với từ hiện tại.
              if (candidates_len>nid + wid):
                candidates[nid + wid].update([word])
      # Xây dựng câu đúng bằng cách chọn từ phổ biến nhất từ mỗi bảng ứng viên từ.
      #output = ' '.join(c.most_common(1)[0][0] for index, c in enumerate(candidates))
      output = ' '.join(c.most_common(1)[0][0] if c else sentence.split()[index] for index, c in enumerate(candidates))

    return output

def check_text(regex, text):
    pattern = re.compile(regex)
    if pattern.fullmatch(text):
        return True
    else:
        return False
def correct_2(text):
  ep = extract_phrases(text)
  rs_ = ""
  for value in ep:
    regex = r'\w[\w ]+'
    if check_text(regex, value):
      rs_ = rs_ + correct(value)
    else:
      rs_ = rs_ + value
  return rs_

In [ ]:
text= "Bất cuws lúc nao, trong cuộc phieeu lưu đơn thương độc max. Tôi ddi học ở đaau. Cơn mưa ngang qua,  cơn muwa  @ ngang qua\nBất cuws lúc nao, trong cuộc phieeu lưu đơn thương độc max. Tôi ddi học ở đaau. Cơn mưa ngang qua,  cơn muwa  @ ngang qua"
results = correct_2(text)
results

'Bất cứ lúc nao, trong cuộc phiêu lưu đơn thương độc mã. Tôi đi học ở đâu. Cơn mưa ngang qua,  cơn mưa  @ ngang qua\nBất cứ lúc nao, trong cuộc phiêu lưu đơn thương độc mã. Tôi đi học ở đâu. Cơn mưa ngang qua,  cơn mưa  @ ngang qua'

In [ ]:
import pickle

# Đọc dữ liệu từ file
with open('data/my_data_test.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

print("Data loaded successfully!")
print(loaded_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(loaded_data)

274155

In [ ]:
test_data = loaded_data[0:2000]
print(test_data[0])

[' Mạo hiểm rừng Đa Mi Cuộc hành quân khám phá thác sương mù Ẩn mình trong cánh rừng Đa Mi (Hàm Thuận Bắc, Bình Thuận) bạt ngàn là hồ thủy điện Đa Mi đẹp như một nàng công chúa, ngọn thác sương mù hùng vĩ, cao ngất, uốn lượn như con rồng bạc khổng lồ', ' Mạo ihểm rừng Đa Mi Cuộc hành quân khá mphá thác sương mù Ẩn mình trong cánh rừng Đa Mi (Hàm Thuận Bắc, Bình Thuận) bạt ngàn là hồ thủy điện Đa Mi đẹp như một nàng công chúa, ngọn thác sương mù hùng vĩ, cao ngất, uốn qượn như con rồng bạc khổng lồ']


In [ ]:
correct_2("000 khách đến vịnh Nha Trang Theo trực ban bộ đội biên phòng tại cảng du lịch Cầu Đá (Vĩnh Nguyên), trong những ngày lễ vừa qua có hơn 16")

'000 khách đến vịnh Nha Trang Theo trực ban bộ đội biên phòng tại cảng du lịch Cầu Đá (Vĩnh Nguyên), trong những ngày lễ vừa qua có hơn 16'

In [ ]:
correct_2("Lễ té nước, người Thái ọgi la Song Kian - tương tự Tết Nguyên đán của người Việt - bắt đầu từ ngày 12 đến 14-")

'Lễ té nước, người Thái gọi la Song Kian - tương tự Tết Nguyên đán của người Việt - bắt đầu từ ngày 12 đến 14-'

In [12]:
from tqdm import tqdm
result_predict = []
result_predict_row_error = []
result_predict_real = []
for text in tqdm(test_data):
    # Kiểm tra nếu cụm từ không khớp với mẫu ký tự cho phép thì bỏ qua
    results = correct_2(text[1])
    result_predict.append(results)
    result_predict_real.append(text[0])
    if(results != text[0]):
      result_predict_row_error.append([text[0], results])



100%|██████████| 2000/2000 [1:02:05<00:00,  1.86s/it]


In [13]:
len(result_predict_row_error)

1115

In [14]:
import string
import unicodedata
import editdistance


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))

        pd_cer, gt_cer = list(pd.lower()), list(gt.lower())
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))

        pd_wer, gt_wer = pd.lower().split(), gt.lower().split()
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))

        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))

    cer_f = sum(cer) / len(cer)
    wer_f = sum(wer) / len(wer)
    ser_f = sum(ser) / len(ser)

    return (cer_f, wer_f, ser_f)

In [15]:
test_data_noise = []
for item in test_data:
  test_data_noise.append(item[1])

In [16]:
evaluate = ocr_metrics(predicts=test_data_noise,
                                  ground_truth=result_predict_real,
                                  norm_accentuation=False,
                                  norm_punctuation=False)

e_corpus = "\n".join([
    "Metrics:",
    "Character Error Rate: {}".format(evaluate[0]),
    "Word Error Rate:      {}".format(evaluate[1]),
    "Sequence Error Rate:  {}".format(evaluate[2]),
])

In [17]:
print(e_corpus)

Metrics:
Character Error Rate: 0.040929927754441243
Word Error Rate:      0.11968599781415802
Sequence Error Rate:  0.8625


In [18]:
evaluate = ocr_metrics(predicts=result_predict,
                                  ground_truth=result_predict_real,
                                  norm_accentuation=False,
                                  norm_punctuation=False)

e_corpus = "\n".join([
    "Metrics:",
    "Character Error Rate: {}".format(evaluate[0]),
    "Word Error Rate:      {}".format(evaluate[1]),
    "Sequence Error Rate:  {}".format(evaluate[2]),
])

In [19]:
print(e_corpus)

Metrics:
Character Error Rate: 0.01196918042536294
Word Error Rate:      0.043079523969120695
Sequence Error Rate:  0.5575
